In [1]:
# changing the working directory 
import os
print("Current working directory:", os.getcwd()) 
os.chdir("C:/Users/mgabi/Desktop/immo-eliza-ml") ############ specify your working directory ############

Current working directory: c:\Users\mgabi\Desktop\immo-eliza-ml\Notebooks


In [2]:
import numpy as np
import pandas as pd
file_path =r"Cleaning\properties.csv"
data = pd.read_csv(file_path)




In [3]:
#some insights about the data
print(f"Total rows before cleaning: {len(data)}")
data.describe
data.info

Total rows before cleaning: 75511


<bound method DataFrame.info of              id     price property_type    subproperty_type            region  \
0      34221000  225000.0     APARTMENT           APARTMENT          Flanders   
1       2104000  449000.0         HOUSE               HOUSE          Flanders   
2      34036000  335000.0     APARTMENT           APARTMENT  Brussels-Capital   
3      58496000  501000.0         HOUSE               HOUSE          Flanders   
4      48727000  982700.0     APARTMENT              DUPLEX          Wallonia   
...         ...       ...           ...                 ...               ...   
75506  30785000  210000.0     APARTMENT           APARTMENT          Wallonia   
75507  13524000  780000.0     APARTMENT           PENTHOUSE  Brussels-Capital   
75508  43812000  798000.0         HOUSE  MIXED_USE_BUILDING  Brussels-Capital   
75509  49707000  575000.0         HOUSE               VILLA          Flanders   
75510  65278000  515000.0     APARTMENT           PENTHOUSE          Flanders

In [4]:
#Removing unnecessarily columns #
data.drop(columns=[ 'cadastral_income', 'primary_energy_consumption_sqm'], inplace=True)
data.columns

Index(['id', 'price', 'property_type', 'subproperty_type', 'region',
       'province', 'locality', 'zip_code', 'latitude', 'longitude',
       'construction_year', 'total_area_sqm', 'surface_land_sqm',
       'nbr_frontages', 'nbr_bedrooms', 'equipped_kitchen', 'fl_furnished',
       'fl_open_fire', 'fl_terrace', 'terrace_sqm', 'fl_garden', 'garden_sqm',
       'fl_swimming_pool', 'fl_floodzone', 'state_building', 'epc',
       'heating_type', 'fl_double_glazing'],
      dtype='object')

In [5]:
#Calculating the missing data for each column and its percentage
missing_values = data.isna().sum()  
total_rows = len(data)
missing_percentage = round(((missing_values / total_rows) * 100),2)
missing_data = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percentage
})

print(missing_data)


                   Missing Values  Percentage
id                              0        0.00
price                           0        0.00
property_type                   0        0.00
subproperty_type                0        0.00
region                          0        0.00
province                        0        0.00
locality                        0        0.00
zip_code                        0        0.00
latitude                    14098       18.67
longitude                   14098       18.67
construction_year           33391       44.22
total_area_sqm               7615       10.08
surface_land_sqm            36256       48.01
nbr_frontages               26346       34.89
nbr_bedrooms                    0        0.00
equipped_kitchen                0        0.00
fl_furnished                    0        0.00
fl_open_fire                    0        0.00
fl_terrace                      0        0.00
terrace_sqm                 13140       17.40
fl_garden                       0 

In [6]:
# REmoving duplication only of the id and zip_codes matches
data_cleaned = data.drop_duplicates(subset=['id', 'zip_code'], keep=False)
print(f"Total rows after cleaning: {len(data_cleaned)}") # printing the number of observatin after cleaning

# Remove leading, trailing ,and inside spaces from all string columns 
data_cleaned_spaces = data_cleaned.apply(lambda x: x.str.strip() if x.dtype == "object" else x)  # using .str.strip() this is only to remove the head and tail spaces
data_cleaned_spaces = data_cleaned.apply(lambda x: x.str.replace(r'\s+', ' ', regex=True) if x.dtype == "object" else x) # x.str.replace(r'\s+', ' ', regex=True) using regex to remove unwanted spaces between words.
 
# Filling the empty cells with 'NaN' if it is numeric and 'None' if it is 'string'  to be encoded later in the preprocessing step
data_cleaned_spaces.replace({'MISSING': None, '': None}, inplace=True)
def fill_missing_values(col):
    if col.dtype == 'object':  # Check if the column is of type object (string)
        return col.where(col.notna(), None)  
    else:  # For numeric columns
        return col.where(col.notna(), np.nan)  
data_cleaned_final_1 = data_cleaned_spaces.apply(fill_missing_values)

#Solving the encoding errors for price if it is exists to avoid any errors that can be occured 
data_cleaned_final_1['price'] = data_cleaned_final_1['price'].replace({'\€': '', ',': ''}, regex=True)

#removing raws with empty values  
columns_to_check = ['province', 'zip_code','region','id','price','locality']  # Columns to check for empty values
data_cleaned_final = data_cleaned_final_1.dropna(subset=columns_to_check, how='any') 

''' remove rows with empty strings. This is for confirmation, 
# it can be useless as explained in the previous 2 comments".
#however it has to be applied if I used it before conversion '''
data_cleaned_final = data_cleaned_final[(data_cleaned_final[columns_to_check] != '').all(axis=1)] 
print(f"Total rows after cleaning: {len(data_cleaned_final)}")


print( data_cleaned_final.dtypes)
#Saving the cleaned file in the choesen path.
data_cleaned_final.to_csv(r"Cleaning\properties_cleaned.csv" ,index=False)

Total rows after cleaning: 75221
Total rows after cleaning: 75218
id                     int64
price                float64
property_type         object
subproperty_type      object
region                object
province              object
locality              object
zip_code               int64
latitude             float64
longitude            float64
construction_year    float64
total_area_sqm       float64
surface_land_sqm     float64
nbr_frontages        float64
nbr_bedrooms         float64
equipped_kitchen      object
fl_furnished           int64
fl_open_fire           int64
fl_terrace             int64
terrace_sqm          float64
fl_garden              int64
garden_sqm           float64
fl_swimming_pool       int64
fl_floodzone           int64
state_building        object
epc                   object
heating_type          object
fl_double_glazing      int64
dtype: object
